# Wprowadzenie do systemów rekomendacyjnych

Warsztaty prowadzone w ramach serii Accenture Training Labs 2022.
Celem tego notebooka jest wygenerowanie rekomendacji dla Klienta indywidualnego w ramach dwóch Use Casów - prostych rekomendacji i rekomendacji spersonalizowanych.


### Instrukcja uzupełniania
Miejsca gdzie należy napisać/uzupełnić kod znajdują się bezpośrednio **pod** komentarzami oznaczonymi jako `TODO`.

Przykład:
```python
a = some_function()
# TODO: stwórz nową zmienną z napisem 'zmienna'
b = ...
# TODO: wypisz zmienną b
...
```
Rozwiązanie:
```python
a = some_function()
# TODO: stwórz nową zmienną z napisem 'zmienna'
b = 'zmienna'
# TODO: wypisz zmienną b
print(b)
```
Jeśli wymagane jest odpowiednie nazewnictwo zmiennych będą one zadeklarowane z przypisanym operatorem ```...``` (tak jak jest to pokazane powyżej).

Jeśli nie uda Ci się wykonać zadań opisanych w `TODO`, na samym dole notebooka znajduje się ściąga pozwalająca uzupełnić brakujące linijki i kontynuować wasztat.

### Importy bibliotek

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from typing import Union
from IPython.display import display
import os
from pathlib import Path
from statsmodels.tsa.seasonal import seasonal_decompose

Ustawienie seed dla zachowania powtarzalności wyników.

In [ ]:
np.random.seed(50)

### Wczytanie danych

In [ ]:
fileName = "WarsztatyATLRekomendacjeFixed.csv"

# ścieżki dostępu do plików
dataFolderPath = os.path.join(os.path.pardir, "data")

dataRawPath = os.path.join(dataFolderPath, fileName)

dataRaw = pd.read_csv(dataRawPath, parse_dates=True, encoding="unicode_escape")

### Cheat Sheet:
1. Wczytanie danych do DataFrame'u z pliku *.csv* z zastosowaniem kodowania:
    ```python
    df = pd.read_csv("ścieżka_do_pliku.csv", encoding="unicode_escape")
    ```


2. Wyświetlanie `n` pierwszych wierszy DataFrame'u (`n` domyślnie równe 5):
    * jeśli to ostatnia komenda w komórce:
    ```python
    df.head(n)
    ```
    * jeśli tak nie jest:
    ```python
    display(df.head(n))
    ```


3. Obrót DataFrame'u (tzw. *pivot*) sumujący wartości z ```kolumna_do_zliczenia``` we wszystkich kombinacjach     ```kolumna_x``` z ```kolumna_y```:
    ```python
    df = pd.pivot_table(df, values='kolumna_do_zliczania', index='kolumna_x', columns='kolumna_y', aggfunc=np.sum)
    ```


4. Zsumowanie wartości z danej kolumny `kolumna_wartości` względem `kolumna_grupująca`:
    ```python
    df.groupby("kolumna_grupująca")["kolumna_wartości"].sum()
    ```


5. Sortowanie malejące wierszy w tabeli DataFrame według wartości w kolumnie/kolumnach:
    * dla jednej kolumny
    ```python
    df = df.sort_values('kolumna', ascending=False)
    ```
    * dla wielu kolumn
    ```python
    df = df.sort_values(['kolumna_1', 'kolumna_2'], ascending=False)
    ```


6. Usuwanie duplikatów wierszy na podstawie wartości w kolumnie:
    ```python
    df = df.drop_duplicates('nazwa_kolumny', keep='first')
    ```


7. Uzyskiwanie wartości minimalnej / maksymalnej  w kolumnie:
    ```python
    df["kolumna"].min() / df["kolumna"].max()
    ```


8. Zliczenie częstotliwości występowania wartości w kolumnie:
    ```python
    df["kolumna"].value_counts()
    ```


9. Otrzymywanie zbioru podstawowych statystyk dotyczących danej kolumny:
    ```python
    df["kolumna"].describe()
    ```


10. Przedstawianie wynikowej serii na wykresie:
    ```python
    plt.plot(seria)
    ```
    
11. Filtrowanie DataFrame'u, tak by zawierał tylko rekordy zawierające określoną wartość kolumny:
    ```python
    df[df['kolumna'] == wartość_kolumny] (tylko rekordy, gdzie wartość kolumny jest równa wartość_kolumny) 
    df[df['kolumna'].isin(lista_wartości)] (tylko rekordy, gdzie wartość kolumny znajduje się w liście wartości lista_wartości)
    df[df['kolumna'].str.contains(sub_string) (tylko rekordy, gdzie zawartość kolumny kolumna zawiera sub_string)
    ```
    
12. Negowanie warunków logicznych
    ```python
    df[~(df['kolumna'] == wartość_kolumny)] (tylko rekordy, gdzie wartość kolumny jest różna od wartość_kolumny) 
    ```
    
13. Operacje na indeksie tabeli:
    * ręczne ustawienie indeksu tabeli
    ```python
    df = df.set_index(kolumna)
    ```
    * sortowanie tabeli po indeksie
    ```python
    df = df.sort_index()
    ```

14. Łączenie tabel po kluczu
    ```python
    merged = pd.merge(dataset_1, dataset_2, on = kolumna_klucz, how = rodzaj_łączenia)
    ```
    
15. Warunkowe wypełnianie wartości kolumny
    ```python
    df[nowa_kolumna] = np.where(warunek_logiczny_uzupełniania_wartości, wartość_przy_spełnionym_warunku, wartość_alternatywna) 
    ```

## Use Case 00 - Sprawdźmy jakie dane dostaliśmy od Klienta

#### Szybki przegląd danych - Jak wygląda nasz zbiór danych?

In [ ]:
dataRaw.head(10)

#### Szybki przegląd danych - Jakie kolumny zawiera nasz zbiór danych?

In [ ]:
dataRaw.columns

#### Szybki przegląd danych - Jakie typy danych zawieraja nasz zbiór danych domyślnie?

In [ ]:
dataRaw.dtypes

#### Szybki przegląd danych - Czy w zbiorze danych występują nulle?

In [ ]:
dataRaw.info()

## Zadania 00 - Sprawdźmy co zawierają nasze dane

In [ ]:
def load_parse_data(file_path):
    """Funkcja wczytuje i przetwarza dane, usuwa kolumny zawierające zerowe wartości oraz
    określa rodzaj danych dla poszczególnych kolumn."""
    global df
    if file_path:
        df = pd.read_csv(file_path, encoding="unicode_escape")
    elif df is ...:
        raise ValueError("The DataFrame cannot be equal to ..., you need to load the data first!")
    df = df.dropna()
    df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
    df["Quantity"] = df.Quantity.astype("int32")
    df["UnitPrice"] = df.UnitPrice.astype("float32")
    df["CustomerID"] = df.CustomerID.astype("int16")
    df["Country"] = df.Country.astype("category")
    
    return df

#### Zadanie 0.1
Wczytaj dane do DataFrame'u oraz zaprezentuj początkowe wiersze.

In [ ]:
# TODO: Zdefiniuj zmienną "df_path" tak, aby zawierała ścieżkę do danych. 
# Możesz wykorzystać zmienną definiowaną wcześniej lub podać ręcznie ścieżkę do danych.
df_path = ...

df = load_parse_data(df_path)

# TODO: wyświetl 10 pierwszych wierszy DataFramu "df"
...

#### Zadanie 0.2
Bardzo często przychodzi nam w pracy konieczność klasyfikacji klientów. Na początek, zobaczmy czy osoby w zbiorze danych to klienci indywidualni czy biznesowi. Najłatwiej będzie to sprawdzić przez sprawdzenie krotności produktów kupowanych przez klientów.

In [ ]:
# TODO: Zdefiniuj zmienną "top_10_quantities" dla wyświetlenia pierwszych dziesięciu najczęściej spotykanej krotności produktów
top_10_quantities = ...
top_10_quantities.sort_index(inplace=True)
top_10_quantities

In [ ]:
# TODO: Zdefiniuj zmienną "top_10_quantities" dla wyświetlenia pierwszych dziesięciu najczęściej spotykanej krotności produktów. 
# Wynik przedstaw w procentach.
top_10_quantities = df.Quantity.value_counts(...).head(10)
top_10_quantities.sort_index(inplace=True)
top_10_quantities

#### Zadanie 0.3
Bądź czujny! Jeśli dane nie zawierają niespodzianek, to znaczy, że jeszcze ich nie odkryłeś. Sprawdźmy jak przedstawia się cały zakres ilości dla wszystkich produktów.

In [ ]:
# TODO: Sprawdź zakres wartości kolumny "Quantity"
df_quantity_min = ...
df_quantity_max = ...
print(df_quantity_min)
print(df_quantity_max)

In [ ]:
df.describe()

##### TODO: Jak myślisz, czemu w danych występują wartości ujemne?

#### Zadanie 0.4
Upewnijmy się, że rzeczywiście ujemna liczba produktów znajduje pokrycie w uprzednich zamówieniach klienta.

In [ ]:
is_lesseq_0_quant_cust = (df.groupby("CustomerID")["Quantity"].sum() <= 0)
customers_with_negative_product_quantity = is_lesseq_0_quant_cust[is_lesseq_0_quant_cust].index
# TODO: Wyświetl zamówienia pierwszego Klienta z listy customers_with_negative_product_quantity 
# i sprawdź czy zamówienie anulowane zawiera produkty, które zostały wcześniej kupione
df[df.CustomerID == ...]

Tak jak przewidywaliśmy, są tutaj wycofane zamówienia. Litera `C` w numerze rachunku najpewniej oznacza "Cancelled" (anulowane).

#### Zadanie 0.5
Ważnym aspektem w pracy z danymi jest sprawdzenie przedziału czasu jakiego dotyczą dane. 

In [ ]:
# TODO: Sprawdź ile z jakiego okresu pochodzą dane udostępnione przez Klienta 
# (liczba dni między pierwszym i ostatnim zakupem)
(... - ...).round('D')

#### Zadanie 0.6
Połowa za nami. Wejdźmy na wyższy poziom! Co powiesz na dodawanie kolumn do DataFrame'u? W tym przypadku, dodaj jednolitą datę rachunku oraz kwotę sprzedaży dla danego wiersza. Następnie, przedstaw wykres obrazujący swoje wyniki!

In [ ]:
def plot_sales(sales_per_week):
    plt.figure(figsize=(15,3))
    labels = np.array([f"Y{t[0] % 2000} W{t[1]:02}" for t in sales_per_week.index])
    n_values = len(sales_per_week.values)
    plt.plot(range(n_values), sales_per_week.values)
    spread = np.arange(n_values, step=n_values // 13)
    plt.xticks(spread, labels[spread])
    plt.title("Sales per Week")

In [ ]:
# TODO: dodaj trzy nowe kolumny do DataFrame'u: rok i tydzień rachunku oraz
# całkowitą kwotę sprzedaży per wiersz
yearWeekDay = df.InvoiceDate.dt.isocalendar()
df["InvoiceWeek"] = ....week
df["InvoiceYear"] = ....year
df["ItemTotalAmount"] = ... * ...
sales_per_week = df.groupby(["InvoiceYear", "InvoiceWeek"])["ItemTotalAmount"].sum()
# TODO: pokaż wykresy przedstawiające sprzedaż w czasie
# następnie oceń czy w danych występuje sezonowość 
plot_sales(...)


#### Zadanie 0.7
Często, będąc analitykiem, możesz mieć potrzebę uzyskiwać podstawowe statystyki dot. danych. Dowiedzmy się nieco więcej na temat kolumny `Country` (państwo).

In [ ]:
# TODO: sprawdź ile jest państw w zbiorze danych, które z nich jest
# najczęściej występujące i ile wierszy przynależy do tego kraju
df.....describe()

#### Zadanie 0.8
Bywa, że niekiedy zamiast na ściśle określonych wartościach, bazujemy na kodach, które je reprezentują. W takich przypadkach, by ułatwić zrozumienie sobie i innym warto stworzyć rodzaj przyporządkowania kodu do tejże wartości. Poniżej, utwórz takie przyporządkowanie dla kodu magazynowego (`StockCode`) i pełnej nazwy produktu, albo inaczej jego opisu (`Description`).

In [ ]:
# TODO: Zdefiniuj zmienną "STOCK_DESC_MAP" przez wyjęcię z dataframu kolumn "StockCode" i "Description", 
# a następnie usuń duplikujące się wiersze. Określ ręcznie indeks tabeli wykorzystując kolumnę "StockCode"
STOCK_DESC_MAP = df[[..., ...]].drop_duplicates(subset=["StockCode"]).set_index(...)

STOCK_DESC_MAP = STOCK_DESC_MAP.squeeze()

# Use Case 1 - Proste rekomendacje

### Sub Use Case 1 - Rekomendowanie produktów najczęściej kupowanych w zadanym przedziale czasu

In [ ]:
# Znalezienie daty ostatniego zakupu
today = df['InvoiceDate'].max()

In [ ]:
def find_the_most_popular_products_in_time_period(number_of_days_taken_into_period, the_latest_date):
    
    most_popular_products_in_time_period = df[(df['InvoiceDate'] <= the_latest_date) & (df['InvoiceDate'] >= the_latest_date - pd.Timedelta(days=number_of_days_taken_into_period))]\
    .groupby("Description")\
    .size()\
    .reset_index(name='counts')\
    .sort_values("counts",ascending=False)\
    .head(5)
    
    top_recommendations = most_popular_products_in_time_period['Description'].tolist()
    
    return most_popular_products_in_time_period, top_recommendations

In [ ]:
number_of_days_taken_into_period = 7
most_popular_products_in_time_period, top_recommendations = find_the_most_popular_products_in_time_period(number_of_days_taken_into_period, today)

print("Produkty najczęściej kupowane w ciągu ostatnich {} dni:".format(number_of_days_taken_into_period))
print(most_popular_products_in_time_period)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 12583")
print(top_recommendations)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 17420")
print(top_recommendations)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 17760")
print(top_recommendations)
print("-----------------------------------------------")

In [ ]:
number_of_days_taken_into_period = 30
most_popular_products_in_time_period, top_recommendations = find_the_most_popular_products_in_time_period(number_of_days_taken_into_period, today)

print("Produkty najczęściej kupowane w ciągu ostatnich {} dni:".format(number_of_days_taken_into_period))
print(most_popular_products_in_time_period)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 12583")
print(top_recommendations)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 17420")
print(top_recommendations)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 17760")
print(top_recommendations)
print("-----------------------------------------------")

#### Zadanie 1 - Znalezienie produktów najczęściej anulowanych/reklamowanych

In [ ]:
# Dostosuj funkcję tak, aby zwracała 20 najczęściej zwracanych produktów w zadanym okresie
def find_the_most_cancelled_products_in_time_period(number_of_days_taken_into_period, the_latest_date):
    
    # TODO: Uzupełnij warunki filtrowania tak, aby df_canceled zawierało wyłącznie anulowane zamówienia
    # TODO - Opcja 1 - Podejście związane z filtrowaniem InvoiceNo - musi zawierać literę "C"
    df_canceled = df[...]    
    
    # TODO - Opcja 2 - Podejście związane z filtrowaniem Quantity - musi być ujemne
    df_canceled = df[...] 
    
    most_cancelled_products_in_time_period = df_canceled[(df_canceled['InvoiceDate'] <= the_latest_date) & (df_canceled['InvoiceDate'] >= the_latest_date - pd.Timedelta(days=number_of_days_taken_into_period))]\
    .groupby("Description")\
    .size()\
    .reset_index(name='counts')\
    .sort_values("counts",ascending=False)\
    .head(20)
    
    blacklist = most_cancelled_products_in_time_period['Description'].tolist()
    
    return most_cancelled_products_in_time_period, blacklist

In [ ]:
today = df['InvoiceDate'].max()
period_days = 30

# TODO: Uzupełnij parametry funkcji tak, aby uwzględniała liczbę dni podaną w zmiennej "period_days" oraz ostatnią datę zakupu zawartą w zmiennej "today"
most_cancelled_products_in_time_period, blacklist = find_the_most_cancelled_products_in_time_period(..., ...)

print("Produkty najczęściej zwracane w ciągu ostatnich {} dni:".format(number_of_days_taken_into_period))
print(most_cancelled_products_in_time_period)

print("-----------------------------------------------")
print("Lista 20 najcześciej zwracanych produktów:")
print(blacklist)

#### Zadanie 2 - Rekomendowanie produktów najczęściej kupowanych pomijając produkty najczęściej zwracane/anulowane/reklamowane

In [ ]:
def find_the_most_popular_products_in_time_period_include_blacklist(number_of_days_taken_into_period, the_latest_date, blacklist):
    
    # TODO 1: Uzupełnij brakującą nazwę kolumny, którą powinniśmy wykorzystać do odfiltrowania numerów produktów zgodnie z produktami podanymi w parametrze "blacklist" 
    # TODO 2: Uzupełnij parametr metody "isin" tak, aby wykluczyć produkty znajdujące się na liście podanej w parametrze funkcji
    df_after_blacklist_drop = df[....isin(...)]
    
    # TODO: Uzupełnij brakującą nazwę kolumny, którą powinniśmy wykorzystać do odfiltrowania odpowiednich dat, uwzględniając parametry "the_latest_date" oraz "number_of_days_taken_into_period"
    most_popular_products_in_time_period = df_after_blacklist_drop[(df_after_blacklist_drop[...] <= the_latest_date) & (df_after_blacklist_drop[...] >= the_latest_date - pd.Timedelta(days=number_of_days_taken_into_period))]\
    .groupby("Description")\
    .size()\
    .reset_index(name='counts')\
    .sort_values("counts",ascending=False)\
    .head(5)
    
    top_recommendations = most_popular_products_in_time_period['Description'].tolist()
    
    return most_popular_products_in_time_period, top_recommendations

In [ ]:
today = df['InvoiceDate'].max()
number_of_days_taken_into_period = 30

#TODO: Uzupełnij parametry funkcji find_the_most_popular_products_in_time_period_include_blacklist tak by uwzględniała 
# zmienne "today", "number_of_days_taken_into_period" oraz "blacklist"
most_popular_products_in_time_period, top_recommendations = find_the_most_popular_products_in_time_period_include_blacklist(..., ..., ...)

print("Produkty najczęściej kupowane (uwzględniając produkty anulowane) w ciągu ostatnich {} dni:".format(number_of_days_taken_into_period))
print(most_popular_products_in_time_period)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 12583")
print(top_recommendations)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 17420")
print(top_recommendations)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 17760")
print(top_recommendations)
print("-----------------------------------------------")

#### Czy rekomendacje uległy zmianie?

In [ ]:
number_of_days_taken_into_period = 30
most_popular_products_in_time_period, top_recommendations = find_the_most_popular_products_in_time_period(number_of_days_taken_into_period, today)

print("Produkty najczęściej kupowane w ciągu ostatnich {} dni:".format(number_of_days_taken_into_period))
print(most_popular_products_in_time_period)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 12583")
print(top_recommendations)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 17420")
print(top_recommendations)

print("-----------------------------------------------")
print("Rekomendacje dla klienta 17760")
print(top_recommendations)
print("-----------------------------------------------")

### Sub Use Case 2 - Rekomendowanie produktów trendujących 

In [ ]:
def calculate_purchases_in_time_period(df,the_latest_date, period_start, period_end):
    
    df = df[(df['InvoiceDate'] <= the_latest_date - pd.Timedelta(days=period_start)) & (df['InvoiceDate'] >= the_latest_date - pd.Timedelta(days=period_end))]\
    .groupby("Description")\
    .size()\
    .reset_index(name='counts_{}_{}_days'.format(period_start, period_end))\
    .sort_values('counts_{}_{}_days'.format(period_start, period_end),ascending=False)
    
    return df

def calculate_period_to_period_ratio(counts_period_1, counts_period_2, period_1_start, period_1_end, period_2_start, period_2_end):
    
    merged_periods = pd.merge(counts_period_1, counts_period_2 ,on='Description', how='left')
    merged_periods['period_to_period_ratio'] = merged_periods['counts_{}_{}_days'.format(period_1_start, period_1_end )] / merged_periods['counts_{}_{}_days'.format(period_2_start, period_2_end)]
    
    return merged_periods

def find_trending_products(df, trending_threshold, period_1_start, period_1_end):
    
    df = df[df['period_to_period_ratio']>trending_threshold]\
    .sort_values('counts_{}_{}_days'.format(period_1_start, period_1_end ), ascending=False)\
    .head(10)
    
    return df

In [ ]:
period_1_start, period_1_end = 0 , 7
period_2_start, period_2_end = 8 , 30

counts_period_1 = calculate_purchases_in_time_period(df,today, period_1_start, period_1_end)
counts_period_2 = calculate_purchases_in_time_period(df,today, period_2_start, period_2_end)

comparison_df = calculate_period_to_period_ratio(counts_period_1, counts_period_2, period_1_start, period_1_end, period_2_start, period_2_end)

trending_products = find_trending_products(comparison_df, 2.0, period_1_start, period_1_end)

In [ ]:
trending_products

#### Zadanie 3 - Sprawdź czy produkty z punktu powyżej były sprzedawane wcześniej, czy zostały dopiero wprowadzone do oferty

In [ ]:
def calculate_if_sold_before(df, trending_products, the_latest_date, period_end):
    
    earliest_purchases_per_product = df.groupby('Description')['InvoiceDate'].min().reset_index(name='earliest_purchase')
    earliest_purchases_per_product = earliest_purchases_per_product[['Description', 'earliest_purchase']]
    
    # TODO: Uzupełnij brakujące pola pd.merge() zgodnie z dokumentacją metody pd.merge(dataset1, dataset2, klucz, metoda)
    # TODO 1: Uzupełnij brakujące nazwy datasetów, które musimy połączyć aby otrzymać informacje o produktach trencujących dodając informację o pierwszej dacie zakupu
    # TODO 2: Uzupełnij jaka kolumna może być kluczem dla łączenia danych z dataframów trending_products i earliest_purchases_per_product
    trending_products_with_earliest_purchase = pd.merge(..., ..., on=..., how='left')
    trending_products_with_earliest_purchase['period_end'] = the_latest_date - pd.Timedelta(days=period_end)
    
    # TODO: Uzupełnij warunkowe określenie wartości kolumny "was_product_available_before" tak, aby przyjmowała ona wartości "yes" / "no"
    # Jeśli pierwszy zakup produktu wystąpił przed końcem okresu uwzględnianego w obliczeniach 
    trending_products_with_earliest_purchase['was_product_available_before'] = np.where(...,
                                                                                        ...,
                                                                                        ...)

    return trending_products_with_earliest_purchase

In [ ]:
trending_recommendations = calculate_if_sold_before(df,trending_products, today, period_2_end)
trending_recommendations

In [ ]:
trending_recommendations_without_new_produts = trending_recommendations[trending_recommendations['was_product_available_before'] == 'yes']
trending_recommendations_without_new_produts

### Sub Use Case 3 - Rekomendacje uwzględniające segmentację Klientów

RFM - Recency, Frequency, Monetary

In [ ]:
df['order_value'] = df['Quantity'] * df['UnitPrice']
df['today'] = today
df['time_difference_in_days'] = (df['today'] - df['InvoiceDate'])/ pd.Timedelta(days=1)

In [ ]:
# order_id
rfm_values = df.groupby("CustomerID").agg({"order_value":'sum', "InvoiceNo":pd.Series.nunique, "time_difference_in_days":min}).reset_index()
rfm_values.columns = ['CustomerID', 'customer_value', 'customer_frequency', 'customer_recency']

In [ ]:
def calculate_quartiles_values(df,columns_list, quartiles_list):
    
    quartiles_dict = {}
    
    for column in columns_list:
        for quartile in quartiles_list:
            quartiles_dict["{}_{}".format(column,quartile)] = df[column].quantile(quartile)
            
    return quartiles_dict

In [ ]:
columns_used_for_RFM = ['customer_value', 'customer_frequency', 'customer_recency']
quartiles_list = [0.33, 0.66]

quartiles_values = calculate_quartiles_values(rfm_values, columns_used_for_RFM, quartiles_list)

In [ ]:
recency_conditions = [
    (rfm_values['customer_recency'] <= quartiles_values['customer_recency_0.33']),
    (quartiles_values['customer_recency_0.66'] > rfm_values['customer_recency']) & (rfm_values['customer_recency']>= quartiles_values['customer_recency_0.33']),
    (rfm_values['customer_recency'] > quartiles_values['customer_recency_0.66'])]
recency_choices = [100, 200, 300]

frequency_conditions = [
    (rfm_values['customer_frequency'] >= quartiles_values['customer_frequency_0.66']),
    (quartiles_values['customer_frequency_0.66'] > rfm_values['customer_frequency']) & (rfm_values['customer_frequency']>= quartiles_values['customer_frequency_0.33']),
    (rfm_values['customer_frequency'] < quartiles_values['customer_frequency_0.33'])]
frequency_choices = [10, 20, 30]

value_conditions = [
    (rfm_values['customer_value'] >= quartiles_values['customer_value_0.66']),
    (quartiles_values['customer_value_0.66'] > rfm_values['customer_value']) & (rfm_values['customer_value']>= quartiles_values['customer_value_0.33']),
    (rfm_values['customer_value'] < quartiles_values['customer_value_0.33'])]
value_choices = [1, 2, 3]

rfm_values['recency_segment'] = np.select(recency_conditions, recency_choices)
rfm_values['frequency_segment'] = np.select(frequency_conditions, frequency_choices)
rfm_values['monetary_segment'] = np.select(value_conditions, value_choices)

rfm_values['RFM_score'] = rfm_values['recency_segment'] + rfm_values['frequency_segment'] + rfm_values['monetary_segment']

In [ ]:
plt.figure(figsize=(20,8))
sns.countplot(data = rfm_values, x='RFM_score')

#### Zadanie 4 - Wygeneruj rekomendacje typu "Top 5 najpopularniejszych produktów" uwzględniając Segmenty klientów

In [ ]:
# TODO: Uzupełnij po jakim kluczu możemy połączyć dane z datasetów df oraz rfm_values 
df = pd.merge(df, rfm_values[['CustomerID', 'RFM_score']], on =..., how='left')

In [ ]:
# Ta funkcja znajduje 5 najpopularniejszych produktów uwzględniając zadany przedział czasowy i segment
def find_the_most_popular_products_in_time_period_for_segment(df, number_of_days_taken_into_period, the_latest_date, segment_number):
    
    df = df[df['RFM_score'] == segment_number]
    
    most_popular_products_in_time_period_for_segment = df[(df['InvoiceDate'] <= the_latest_date) & (df['InvoiceDate'] >= the_latest_date - pd.Timedelta(days=number_of_days_taken_into_period))]\
    .groupby("Description")\
    .size()\
    .reset_index(name='counts')\
    .sort_values("counts",ascending=False)\
    .head(5)
    
    top_recommendations_for_segment = most_popular_products_in_time_period_for_segment['Description'].tolist()
    
    return most_popular_products_in_time_period_for_segment, top_recommendations_for_segment


# Ta funkcja przygotowuje rekomendacje dla wybranego klienta, uwzględniając jego segment oraz przedział czasowy
# TODO: Uzupełnij definicję funkcji tak, aby wygenerować rekomendacje dla Klienta od CustomerID określonym przez parametr "customer_id"
def find_recommendations_for_customer(df, number_of_days_taken_into_period, the_latest_date, customer_id):
    # TODO 1: Jaką kolumnę dataframu musimy porównać z parametrem "customer_id" aby znaleźć rekoedy należące tylko do tego Klienta?
    # TODO 2: Jaką kolumną dataframu musimy wyciągnąć, aby przypisać do zmiennej "customer_segment" segment RFM Klienta?
    customer_segment = df[df[...]==customer_id][...].values[0]
    
    most_popular_products_in_time_period_for_segment, top_recommendations_for_segment = find_the_most_popular_products_in_time_period_for_segment(df,number_of_days_taken_into_period, today, customer_segment)
    
    return most_popular_products_in_time_period_for_segment, top_recommendations_for_segment

In [ ]:
number_of_days_taken_into_period = 30
today = df['InvoiceDate'].max()

# TODO: Uzupełnij brakujący parametr funkcji tak, aby otrzymać rekomendacje kolejno dla Klientów o ID: 12583, 17420, 17760
recommendationd_for_customer_1, top_recommendations_for_customer_1 = find_recommendations_for_customer(df,number_of_days_taken_into_period, today, ...)
recommendationd_for_customer_2, top_recommendations_for_customer_2 = find_recommendations_for_customer(df,number_of_days_taken_into_period, today, ...)
recommendationd_for_customer_3, top_recommendations_for_customer_3 = find_recommendations_for_customer(df,number_of_days_taken_into_period, today, ...)

In [ ]:
print("-----------------------------------------------")
print("Rekomendacje uwzględniające segment dla klienta 12583")
print(top_recommendations_for_customer_1)

print("-----------------------------------------------")
print("Rekomendacje uwzględniające segment dla klienta 17420")
print(top_recommendations_for_customer_2)

print("-----------------------------------------------")
print("Rekomendacje uwzględniające segment dla klienta 17760")
print(top_recommendations_for_customer_3)
print("-----------------------------------------------")

# Use Case 2 - Rekomendacje spersonalizowane

Podstawą generowania rekomendacji za pomocą collaborative-filteringu jest ocena podobieństwa między Klientami. Jednym z algorytmów, który nadaje się do wyznaczenia podobieństwa koszyków Klientów zbioru danych jest algorytm k-NN (k Najbliższych Sąsiadów). 
Do jego zastosowania potrzebna jest tabela określająca jakie produkty kupili poszczególni klienci oraz liczba sąsiadów (podobnych klientów).

In [ ]:
def preprocess_data(df):
    """Funkcja wykonuje obrót tabeli, a następnie zlicza brakujące wartości i
    dokonuje ich zaprzeczenia. Rezultat to tabela, określająca czy produkt
    został zakupiony przez danego klienta."""
    
    truth_table = ~df.pivot_table(index="CustomerID",columns="StockCode",values="Quantity").isna()
    
    return truth_table

In [ ]:
truth_table = preprocess_data(df)

Poniżej model generuje odległości od sąsiadów oraz klucze numeryczne dla dwóch najbliższych sąsiadów każdego z klientów.

In [ ]:
def prep_model(df,n_neighbours) :
    """Funkcja przygotowuje model oparty na algorytmie najbliższego sąsiada i
    na jego podstawie wylicza odległości oraz klucze numeryczne dla
    najbliższych sąsiadów wszystkich wartości."""
    
    from sklearn.neighbors import NearestNeighbors
    
    knn = NearestNeighbors(metric="jaccard")
    knn.fit(df)
    distances, indices = knn.kneighbors(df, n_neighbors=n_neighbours)
    
    return distances, indices

In [ ]:
distances, indices = prep_model(truth_table, 3)

Zestaw rekomendacji jest spersonalizowany względem losowo wybranego ID klienta. Jeśli chcesz otrzymać zestaw dla innego klienta, odswież komórkę.

In [ ]:
def prepare_recommendations(customerId, truth_table, indices,):
    """Funkcja przygotowuje rekomendacje w postaci produktów dla konkretnego (indentyfikatora) klienta."""
    cust_idx = truth_table.index.get_loc(customerId)
    closest = list(map(lambda v: truth_table.index[v], indices[cust_idx, 1:]))
    recs = set()
    
    for recommender in closest:
        potential = (truth_table.loc[recommender] ^ truth_table.loc[customerId]) & (~truth_table.loc[customerId])
        potential = potential[potential].index.tolist()
        recs.update(potential)
        
    return recs

def recommend(customerId, truth_table, indices, stock_desc_map,upsell=False):
    """Funkcja przedstawia wygenerowane rekomendacje w postaci tekstowej."""
    recs = prepare_recommendations(customerId, truth_table, indices)
    
    if upsell:
        # odrzucając produkty szczególne, jak opłaty pocztowe
        recs = set(filter(lambda r: not str.isalpha(r), recs))
        # wybierz przeciętnie (medianowo) najdroższy produkt
        recs = [df[df.StockCode.isin(recs)].groupby("StockCode")["UnitPrice"].median().idxmax()]
        
    if recs:
        str_recs = ",\n".join(["* " + stock_desc_map[r] for r in recs])
        return f'You may like the following product(s):\n{str_recs}\nTake a look!'
    else:
        return f"No recommendations available."

In [ ]:
random_cust = np.random.choice(df.CustomerID)
result = recommend(random_cust, truth_table, indices, STOCK_DESC_MAP)
print(result)

#### Zadanie 1 - Wygeneruj rekomendacje dla Klientów o id: 12583,17420,17760. Spojrz jakie produkty są im polecane i oceń poprawność rekomendacji. Czy widzisz sposób na usprawnienie tych rekomendacji?

In [ ]:
selected_customer_ids = [... , ... , ...]

# TODO: Uzupełnij parametry pętli tak, aby wygenerować rekomendacje dla Klientów o id określonych w liście "selected_customer_ids"
for cust_id in ...:
    print(f"Customer {cust_id}:")
    print(recommend(cust_id, truth_table, indices, STOCK_DESC_MAP), "\n")

In [ ]:
print("-----------------------------------------------")
print("Rekomendacje dla klienta 12583")
print(recommend(12583, truth_table, indices, STOCK_DESC_MAP), "\n")

In [ ]:
print("-----------------------------------------------")
print("Rekomendacje dla klienta 17420")
print(recommend(17420, truth_table, indices, STOCK_DESC_MAP), "\n")

In [ ]:
print("-----------------------------------------------")
print("Rekomendacje dla klienta 17760")
print(recommend(17760, truth_table, indices, STOCK_DESC_MAP), "\n")

#### Zadanie 2 - Wygeneruj rekomendacje proponujące up-sell i ocen rekomendacje. W jaki sposób możemy dalej rozwijać rekomendacje spersonalizowane?

In [ ]:
# TODO: wykorzystaj funkcję recommend (zdefiniowaną na kilka komórek wyżej) by
# zastosować up-sell dla klienta z kilkudziesięcioma
# rekomendowanymi produktami
upsold_result = recommend(..., truth_table, indices, STOCK_DESC_MAP, ...)
print(upsold_result)

### Rozwiązania Zadań 00 

1. 
    ```python
    # zrób: wczytaj dane do zmiennej przy użyciu jednej z powyższych funkcji
    df = pd.read_csv("data.csv", encoding="unicode_escape")
    df = load_parse_data()
    # zrób: wyświetl dwanaście pierwszych wierszy
    ```


2. 
    ```python
    # zrób: zdefiniuj poniższą zmienną w celu wyświetlenia pierwszych dziesięciu
    # najczęściej spotykanych ilości produktów
    top_10_quantities = df.Quantity.value_counts().head(10)
    top_10_quantities.sort_index(inplace=True)
    top_10_quantities
    ```


3. 
    ```python
    # zrób: przedstaw zakres ilości produktów w zbiorze danych
    df.Quantity.max(), df.Quantity.min()
    ```


4. 
    ```python
    is_lesseq_0_quant_cust = (df.groupby("CustomerID")["Quantity"].sum() <= 0)
    unusual_customerIds = is_lesseq_0_quant_cust[is_lesseq_0_quant_cust].index
    # zrób: wyświetl zbiór danych dla pierwszego (najmniejszego) względem
    # indentyfikatora klienta spośród nietypowych klientów
    df[df.CustomerID == unusual_customerIds[0]]
    ```


5. 
    ```python
    # zrób: oblicz ile dni zawartych jest w danych
    (df.InvoiceDate.max() - df.InvoiceDate.min()).round('D')
    ```


6. 
    ```python
    # zrób: dodaj dwie nowe kolumny do ramy danych: ujednoliconą datę rachunku
    # (datę rachunku z wyzerowaną częścią czasu) oraz całkowitą kwotę sprzedaży
    # per wiersz
    df["NormalizedInvoiceDate"] = df.InvoiceDate.dt.normalize()
    df["ItemTotalAmount"] = df.Quantity * df.UnitPrice
    sales_per_day = df.groupby("NormalizedInvoiceDate")["ItemTotalAmount"].sum()
    # zrób: stwórz wykres przedstawiający sprzedaż w czasie
    # po wykonaniu wszystkich pozostałych zadań wróć by go upiększyć!
    plt.plot(sales_per_day)
    ```


7. 
    ```python
    # zrób: sprawdź ile jest państw w zbiorze danych, które z nich jest
    # najczęściej występujące i ile wierszy przynależy do tego kraju
    df.Country.describe()
    ```


8. 
    ```python
    # zrób: stwórz tablicę mieszającą pozwalającą na powiązanie kodu
    # magazynowego produktu z jego opisem
    STOCK_DESC_MAP = df[[
        "StockCode", "Description"
    ]].drop_duplicates(subset=["StockCode"]).set_index("StockCode")
    STOCK_DESC_MAP = STOCK_DESC_MAP.squeeze()
    ```

### Rozwiązania Zadań dla Use Case 1 

1. Zadanie 1 - Part 1
    ```python
def find_the_most_cancelled_products_in_time_period(number_of_days_taken_into_period, the_latest_date):
    
    df_canceled = df[df['InvoiceNo'].str.contains('C')]    
    
    df_canceled = df[df['Quantity'] < 0] 
    
    most_cancelled_products_in_time_period = df_canceled[(df_canceled['InvoiceDate'] <= the_latest_date) & (df_canceled['InvoiceDate'] >= the_latest_date - pd.Timedelta(days=number_of_days_taken_into_period))]\
    .groupby("StockCode")\
    .size()\
    .reset_index(name='counts')\
    .sort_values("counts",ascending=False)\
    .head(20)
    
    blacklist = most_cancelled_products_in_time_period['StockCode'].tolist()
    
    return most_cancelled_products_in_time_period, blacklist
    ```
    
2. Zadanie 1 - Part 2
    ```python
    today = df['InvoiceDate'].max()
    period_days = 30

    most_cancelled_products_in_time_period, blacklist = find_the_most_cancelled_products_in_time_period(period_days, today)

    print("Produkty najczęściej zwracane w ciągu ostatnich {} dni:".format(number_of_days_taken_into_period))
    print(most_cancelled_products_in_time_period)

    print("-----------------------------------------------")
    print("Lista 20 najcześciej zwracanych produktów:")
    print(blacklist)
    ```
    
3. Zadanie 2
    ```python
def find_the_most_popular_products_in_time_period_include_blacklist(number_of_days_taken_into_period, the_latest_date, blacklist):
    
    df_after_blacklist_drop = df[~df['StockCode'].isin(blacklist)]
    
    most_popular_products_in_time_period = df_after_blacklist_drop[(df_after_blacklist_drop['InvoiceDate'] <= the_latest_date) & (df_after_blacklist_drop['InvoiceDate'] >= the_latest_date - pd.Timedelta(days=number_of_days_taken_into_period))]\
    .groupby("StockCode")\
    .size()\
    .reset_index(name='counts')\
    .sort_values("counts",ascending=False)\
    .head(5)
    
    top_recommendations = most_popular_products_in_time_period['StockCode'].tolist()
    
    return most_popular_products_in_time_period, top_recommendations
    ```
    ```python
    today = df['InvoiceDate'].max()
    number_of_days_taken_into_period = 30

    #TODO: Uzupełnij parametry funkcji find_the_most_popular_products_in_time_period_include_blacklist tak by uwzględniała 
    # zmienne "today", "number_of_days_taken_into_period" oraz "blacklist"
    most_popular_products_in_time_period, top_recommendations = find_the_most_popular_products_in_time_period_include_blacklist(number_of_days_taken_into_period, today, blacklist)

    print("Produkty najczęściej kupowane (uwzględniając produkty anulowane) w ciągu ostatnich {} dni:".format(number_of_days_taken_into_period))
    print(most_popular_products_in_time_period)

    print("-----------------------------------------------")
    print("Rekomendacje dla klienta 12583")
    print(top_recommendations)

    print("-----------------------------------------------")
    print("Rekomendacje dla klienta 17420")
    print(top_recommendations)

    print("-----------------------------------------------")
    print("Rekomendacje dla klienta 17760")
    print(top_recommendations)
    print("-----------------------------------------------")
    ```
    
4. Zadanie 3
    ```python
def calculate_if_sold_before(df, trending_products, the_latest_date, period_end):
    
    earliest_purchases_per_product = df.groupby('StockCode')['InvoiceDate'].min().reset_index(name='earliest_purchase')
    earliest_purchases_per_product = earliest_purchases_per_product[['StockCode', 'earliest_purchase']]
    
    trending_products_with_earliest_purchase = pd.merge(trending_products, earliest_purchases_per_product, on="StockCode", how='left')
    trending_products_with_earliest_purchase['period_end'] = the_latest_date - pd.Timedelta(days=period_end)
    
    trending_products_with_earliest_purchase['was_product_available_before'] = np.where(trending_products_with_earliest_purchase['earliest_purchase'] < trending_products_with_earliest_purchase['period_end'],
                                                                                        "yes",
                                                                                        "no")
    
    return trending_products_with_earliest_purchase
    ```
    
5. Zadanie 4 - Part 1
    ```python
    df = pd.merge(df, rfm_values[['CustomerID', 'RFM_score']], on ='CustomerID', how='left')
    ```
6. Zadanie 4 - Part 2
    ```python
        def find_the_most_popular_products_in_time_period_for_segment(df, number_of_days_taken_into_period, the_latest_date, segment_number):

            df = df[df['RFM_score'] == segment_number]

            most_popular_products_in_time_period_for_segment = df[(df['InvoiceDate'] <= the_latest_date) & (df['InvoiceDate'] >= the_latest_date - pd.Timedelta(days=number_of_days_taken_into_period))]\
            .groupby("StockCode")\
            .size()\
            .reset_index(name='counts')\
            .sort_values("counts",ascending=False)\
            .head(5)

            top_recommendations_for_segment = most_popular_products_in_time_period_for_segment['StockCode'].tolist()

            return most_popular_products_in_time_period_for_segment, top_recommendations_for_segment

        def find_recommendations_for_customer(df, number_of_days_taken_into_period, the_latest_date, customer_id):
            customer_segment = df[df['CustomerID']==customer_id]['RFM_score'].get_values()[0]

            most_popular_products_in_time_period_for_segment, top_recommendations_for_segment = find_the_most_popular_products_in_time_period_for_segment(df,number_of_days_taken_into_period, today, customer_segment)

            return most_popular_products_in_time_period_for_segment, top_recommendations_for_segment
    ```    
    
7. Zadanie 4 - Part 3
    ```python
    number_of_days_taken_into_period = 30
    today = df['InvoiceDate'].max()

    recommendationd_for_customer_1, top_recommendations_for_customer_1 = find_recommendations_for_customer(df,number_of_days_taken_into_period, today, 12583)
    recommendationd_for_customer_2, top_recommendations_for_customer_2 = find_recommendations_for_customer(df,number_of_days_taken_into_period, today, 17420)
    recommendationd_for_customer_3, top_recommendations_for_customer_3 = find_recommendations_for_customer(df,number_of_days_taken_into_period, today, 17760)

    ```

### Rozwiązania Zadań dla Use Case 2

1. Zadanie 1  
    ```python
    selected_customer_ids = [12583,17420,17760]

    # TODO: Uzupełnij parametry pętli tak, aby wygenerować rekomendacje dla Klientów o id określonych w liście "selected_customer_ids"
    for cust_id in selected_customer_ids:
        print(f"Customer {cust_id}:")
        print(recommend(cust_id, truth_table, indices, STOCK_DESC_MAP), "\n")
    ```


2. Zadanie 2
    ```python
    # TODO: wykorzystaj funkcję recommend (zdefiniowaną na kilka komórek wyżej) by
    # zastosować up-sell dla klienta z kilkudziesięcioma
    # rekomendowanymi produktami
    upsold_result = recommend(12583, truth_table, indices, STOCK_DESC_MAP, upsell= True)
    print(upsold_result)
    ```